In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def transformation(Y):#把male,female转化成0,1,数据清洗
    l=[]
    n = 0
    for y in Y:
        if y =='TLS':
            n = n+1
            l.append(1)
        else:
            l.append(0)
    print(n)
    return np.array(l)
def count(Y):
    n = 0
    for y in Y:
        if y ==1:
            n = n+1
    print(n)

In [3]:
sample = ["c_4","c_3","c_36"]
X_list = []
y_list = []
Gene = []
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#load data
for i in range(len(sample)):
    path = 'result/'+sample[i]+'_annotation.csv'
    data = pd.read_csv(path)
    c = data.shape[0]
    path = "result/"+sample[i]+"_matrix.csv"
    X_data = pd.read_csv(path)
    l_gene = X_data["Unnamed: 0"]
    Gene.append(l_gene)
    bar = X_data.columns[1:]
    l_bar=[]
    for j in bar:
        j=str(j).replace("X","")
        l_bar.append(j)
    a = X_data.shape[0]
    b = X_data.shape[1]-1
    X_data = np.array(X_data[bar])
    X_scaler = scaler.fit(X_data)                                   
    X_data = X_scaler.transform(X_data)                                    
    X_data = pd.DataFrame(X_data.T, columns = l_gene, index = l_bar)
    X_list.append(X_data)
    #-------创建字典----------
    D1 = dict()
    key = []
    for l in range(c):
        x = '-'+str(i)
        key1 = data['Unnamed: 0'][l]
        value1 = data['TLS_2_cat_annot'][l]
        key1 = str(key1).replace('-1','.1')
        key.append(key1)
        D1[key1] = value1
    #-----连接两个数据集,创建输出数据集-------
    L1 = list()
    for j in l_bar:
        if j in key:
            tls = D1[j]
        else:
            tls = 'NO_TLS'
        L1.append(tls)
    y_data = transformation(L1)
    y_data = y_data.ravel()
    y_list.append(y_data)
    

199
257
255


In [4]:
intersection=[]
for i in range(len(Gene)):
    if i==0:
        intersection=set(Gene[i])
    else:
        intersection=set(intersection).intersection(set(Gene[i]))
        
intersection=list(intersection)
for i in range(len(sample)):
    X_data=X_list[i]
    y_data=y_list[i]
    X_data=X_data[intersection]
    X_data = np.array(X_data).tolist()
    y_data = np.array(y_data).tolist()
    if i==0:
        X = X_data
        y = y_data
    else:
        X = X + X_data
        y = y + y_data

X = pd.DataFrame(X)
X = np.array(X)
y = np.array(y)
X = pd.DataFrame(X,columns = intersection)

In [5]:
X_record =X
y_record = y

In [7]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
 
rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)
boruta = BorutaPy(
   estimator = rf, 
   n_estimators = 'auto',
   max_iter = 3 # number of trials to perform
)

# 模型训练
boruta.fit(np.array(X), np.array(y))
# 输出结果
green_area = X.columns[boruta.support_].to_list()
blue_area  = X.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

features in the green area: []
features in the blue area: ['IGKC', 'PPP1R16B', 'TGFBI', 'OGN', 'FCRL1', 'TIMP1', 'IGFBP7', 'FCMR', 'IRF7', 'CTSD', 'FBLN2', 'FTL', 'BANK1', 'BGN', 'CAVIN1', 'COL1A1', 'C3', 'IL32', 'NGFR', 'LGALS3BP', 'MT1E', 'CCN2', 'SPARC', 'HSPB1', 'FSTL1', 'LRP1', 'SELL', 'CST3', 'ACTB', 'AC119396.1', 'IGLC1', 'COL3A1', 'CCDC80', 'MT2A', 'NIBAN3', 'TXNDC5', 'POU2F2', 'CD37', 'COL1A2', 'IGHG1', 'FN1', 'CNN3', 'CD63', 'ITM2B', 'IGHG4', 'ACKR1', 'ACTA2', 'IGHA1', 'CR2', 'IGKV4-1', 'BLK', 'CCL19', 'B2M', 'IGHG2', 'MS4A1', 'JCHAIN', 'CD79A', 'LTB', 'CCL18', 'FDCSP', 'PLA2G2A', 'IGHG3', 'THBS4', 'FCRL3', 'CCL17', 'IGF1', 'PAX5', 'FCGR3A', 'MYL9', 'AEBP1', 'PI16', 'PTGDS', 'IGLV3-1', 'GZMB', 'VIM', 'SULF1']


In [8]:
df = pd.DataFrame(green_area)
df.to_csv("result/green_area_RI.csv")
df = pd.DataFrame(blue_area)
df.to_csv("result/blue_area_RI.csv")